# Modyfikacje formatu danych

In [25]:
import pandas as pd
from pandas import DataFrame, Series

## "Długi" i "szeroki" format danych

W pracy badawczej możemy się zetknąć z (przynajmniej!) dwoma formatami danych. Pierwszy z nich nazwiemy formatem "szerokim", drugi zaś formatem "długim". Bardzo ważną umiejętnością jest umiejętność sprawnego konwertowania jednego formatu do drugiego. Większość bardziej zaawansowanych funkcji statystycznych (regresja liniowa, ANOVA etc.) wymaga danych w formacie "długim".

Formaty te różnią się tym, co reprezentuje jeden wiersz tabeli, innymi słowy - mamy w nich różną jednostkę obserwacji. Aby to zilustrować stwórzmy prosty zbiór danych w formacie "szerokim". Każdy wiesz reprezentuje jednego uczestnika. W każdym wierszu znajdują się informacje o jego liczbie porządkowej, płci oraz ocenie 3 zdań (załóżmy, że każda kolumna reprezentuje odpowiedź na jakieś pytanie lub ocenę jakiegoś zdania)

In [26]:
data = DataFrame({
    'uczestnik' : [1,2,3,4,5,6,7,8],
    'płeć' : ['K','K','M','K','M','M','K','K'],
    'zdanie1' : [6,5,6,6,5,3,4,1],
    'zdanie2' : [4,2,3,6,4,6,4,3],
    'zdanie3' : [1,3,4,5,6,3,5,3]
})
data

,uczestnik,płeć,zdanie1,zdanie2,zdanie3
0,1,K,6,4,1
1,2,K,5,2,3
2,3,M,6,3,4
3,4,K,6,6,5
4,5,M,5,4,6
5,6,M,3,6,3
6,7,K,4,4,5
7,8,K,1,3,3


Format tego rodzaju bardzo często są domyślnym formatem eksportu danych w oprogramowaniu do przeprowadzania badań ankietowych. Załóżmy, że chcemy zmienić nasz format tych danych na format "długi". W takim formacie wiersz nie będzie już reprezentował jednego uczestnika (jak w oryginalnym formacie) ale jedną obserwację rozumianą jako *pojedyncza ocena zdania*. Jednocześnie chcielibyśmy cały czas móc zidentyfikować, od którego uczestnika pochodzą dane oceny.

Aby przejść z formatu "szerokiego" do formatu "długiego" musimy posłużyć się funkcją `melt` z pakietu `pandas`. Funkcja ta przyjmuje następujące argumenty:
- `frame` - tutaj przekazujemy ramkę danych (w przykładzie: `data`)
- `id_vars` - zmienne identyfikujące - tutaj przekazujemy listę nazw kolumn z wartościami, które mają identyfikować nasze obserwacje w nowym formacie (w przykładzie: `płeć` oraz `uczestnik`
- `value_vars` - zmienne z wartościami - tutaj przekazujemy listę nazw kolumn, gdzie znajdują się nasze pomiary (w przykładzie są to `zdanie1`, `zdanie2` oraz `zdanie3`.
- `value_name` - nazwa kolumny z wartościami w nowej ramce danych
- `var_name` - nazwa kolumny ze zmienną w nowej ramce danych

In [27]:
long = pd.melt(data, id_vars =['płeć', 'uczestnik'],
                value_vars = ['zdanie1', 'zdanie2', 'zdanie3'],
                value_name = 'rating',
                var_name = 'zdanie')
long

,płeć,uczestnik,zdanie,rating
0,K,1,zdanie1,6
1,K,2,zdanie1,5
2,M,3,zdanie1,6
3,K,4,zdanie1,6
4,M,5,zdanie1,5
5,M,6,zdanie1,3
6,K,7,zdanie1,4
7,K,8,zdanie1,1
8,K,1,zdanie2,4
9,K,2,zdanie2,2


Łatwo zobaczyć, że "długi" format jest faktycznie długi! 

A co jeśli chcielibyśmy wykonać odwrotną operację? Mamy dane w formacie "długim" i chcemy przekonwerować je do szerokiego? W poniższym przykładzie mamy ramkę danych z obserwacjami z eksperymentu, w którym mierzyliśmy czas reakcji (kolumna `rt`). Pomiar czasu reakcji przeprowadzaliśmy dla każdego badanego trzykrotnie - raz w warunku `C`, drugi raz w warunku `I` i trzeci raz w warunku `N`.

In [28]:
data = DataFrame({
    'participant' : [1,1,1,2,2,2,3,3,3,4,4,4],
    'condition' : ['C', 'I', 'N', 'C', 'I', 'N', 'C', 'I', 'N', 'C', 'I', 'N'],
    'sex' : ['M','M','M','M','M','M','K','K','K','M','M','M'],
    'rt' : [341,234,143,543,231,353,554,321,146,268,764,643]
})
data

,participant,condition,sex,rt
0,1,C,M,341
1,1,I,M,234
2,1,N,M,143
3,2,C,M,543
4,2,I,M,231
5,2,N,M,353
6,3,C,K,554
7,3,I,K,321
8,3,N,K,146
9,4,C,M,268


Aby przekonwertować dane do formatu "szerokiego" należy posłużyć się metodą `pivot` klasy `DataFrame`.
- `index` - tutaj nazwa kolumny, w której wartości mają identyfikować jedną obserwacje, (w przykładzie chcemy 1 obserwacja = 1 uczestnik więc `participant`)
- `columns` - tutaj nazwa kolumny lub kolumn, z których wartości mają być nowymi nazwami kolumn
- `values` - tutaj nazwa kolumny, gdzie przechowujemy wartości

In [29]:
wide = data.pivot('participant', 'condition', 'rt')
wide

condition,C,I,N
participant,,,
1,341,234,143
2,543,231,353
3,554,321,146
4,268,764,643


## Tabulacja danych

Czasami, kiedy badana przez nas zmienna jest zmienną nominalną, chcielibyśmy zagregować nasze dane w postaci tabeli krzyżowej. W takiej tabeli wartościami w komórkach są zliczenia dla poszczególnych rodzajów obserwacji. Taka forma danych będzie niezbędna do zastosowania testów statystycznych takich jak test niezależności $\chi^2$

Załóżmy, że nasze dane wyglądają tak: 

In [31]:
data = DataFrame({
    'participant' : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    'condition' : ['C','E','C','E','C','E','C','E','C','E','C','E','C','E','C','E','C','C','E','C'],
    'response' : ['Y','N','Y','D','N','D','N','Y','Y','D','Y','Y','D','Y','D','N','D', 'D','Y','Y'],
    'sex' : ['K','K','M','M','M','K','M','K','K','M','K','M','K','M','K','M','K','K','M','K']
})
data

,participant,condition,response,sex
0,1,C,Y,K
1,2,E,N,K
2,3,C,Y,M
3,4,E,D,M
4,5,C,N,M
5,6,E,D,K
6,7,C,N,M
7,8,E,Y,K
8,9,C,Y,K
9,10,E,D,M


Chcielibyśmy dowiedzieć się ile było odpowiedzi pozytywnych (`Y`(es)), negatywnych (`N`(o)) i nieokreślonych (`D`(on't know)) w warunku eksperymentalnym (`E`) oraz w grupie kontrolnej (`C`). Aby to zrobić możemy skonstruować tabelę krzyżową poleceniem za pomocą funkcji `crosstab` z pakietu `pandas`:

In [35]:
pd.crosstab(data['response'],data['condition'],
            margins = True) # chcemy również uwzględnić sumy brzegowe

condition,C,E,All
response,,,
D,4,3,7
N,2,2,4
Y,5,4,9
All,11,9,20


Interesującą własnością tego pakietu jest to, że pozwala do tabulacji użyć więcej niż dwóch zmiennych. W tym wypadku jako drugi argument możemy przekazać listę z kolumnami a nie tylko jedną kolumnę jak w przykładzie wyżej:

In [21]:
pd.crosstab(data['response'], [data['sex'], data['condition']])

sex        K     M   
condition  C  E  C  E
response             
D          4  1  0  2
N          0  1  2  1
Y          4  1  1  3

Jeżeli chcielibyśmy zamiast zliczeń otrzymać procenty (tworząc klasyczną tablele rozdzielczą) musimy przekazać odpowiednii argument `normalize`. Wartość `True` oznacza, że wartości we wszystkich komórkach sumują się do 1, wartość `0` oznacza, że wartości wartości sumują się w rzędach do 1, 1 - w kolumnach.

In [37]:
pd.crosstab(data['response'],data['condition'], normalize=True)

condition,C,E
response,,
D,0.20,0.15
N,0.10,0.10
Y,0.25,0.20


In [41]:
pd.crosstab(data['response'],data['condition'], normalize=0)

condition,C,E
response,,
D,0.571429,0.428571
N,0.500000,0.500000
Y,0.555556,0.444444


In [42]:
pd.crosstab(data['response'],data['condition'], normalize=1)

condition,C,E
response,,
D,0.363636,0.333333
N,0.181818,0.222222
Y,0.454545,0.444444
